In [1]:
import torch
import pytorch_lightning as pl
import numpy as np
import matplotlib.pyplot as plt
from pl_celeba import *

print(torch.cuda.is_available())
torch.zeros(1).cuda()
torch.set_float32_matmul_precision('high')

True


In [2]:
def run_case(hyperparams: list) -> None:
    mymodel = CelebAClass(*hyperparams)

    # training
    logger = pl.loggers.tensorboard.TensorBoardLogger('.', name=f'out_{mymodel.datasetname}_{mymodel.epochs}_' + '_'.join([f'{p:.4f}' for p in hyperparams]))
    trainer = pl.Trainer(max_epochs=mymodel.epochs, accelerator='gpu', logger=logger, log_every_n_steps=3)
    trainer.fit(mymodel)

def sweep_hyperparams(hyperparam_list: list, n_runs: int, hyperparams: list = []) -> None:
    if len(hyperparams) == len(hyperparam_list):
        for i in range(n_runs):
            print('-'*80)
            print(f'Running case with hyperparams {hyperparams}')
            print('-'*80)

            run_case(hyperparams)

    else:
        for hyperparam_i in hyperparam_list[len(hyperparams)]:
            new_hyperparams = hyperparams + [hyperparam_i]
            sweep_hyperparams(hyperparam_list, n_runs, new_hyperparams)
    

In [3]:
def main() -> None:
    # model hyperparameters
    cont_var0 = [1e-4]
    cont_lr = [1e-2]
    cont_reg = [0.0]
    warmup_epochs = [1]
    n_runs = 5

    hyperparam_list = [cont_var0, cont_lr, cont_reg, warmup_epochs]
    sweep_hyperparams(hyperparam_list, n_runs)

if __name__ == "__main__":
    main()

--------------------------------------------------------------------------------
Running case with hyperparams [0.0001, 0.01, 0.0, 1]
--------------------------------------------------------------------------------
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./out_CelebA_20_0.0001_0.0100_0.0000_1.0000
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | net  | ResNet | 11.2 M
--------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


Epoch 7:  84%|████████▍ | 966/1145 [01:36<00:17, 10.04it/s, v_num=0, contvar=3.59e-8, train_loss=0.511, val_loss=0.788] 